In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import copy
from sklearn.metrics import mean_squared_error

import ase.io
import equistore.io


# from utils.librascal import RascalSphericalExpansion
from utils.rascaline import RascalineSphericalExpansion
from utils.models.soap import compute_power_spectrum

from utils.models.linear import LinearModel
from utils.models.full_kernel import FullKernelGap
from utils.models.sparse_kernel import SparseKernelGap
from utils.models.sparse_kernel import select_support_points, SparseKernelGap
from utils.pylode import PyLODESphericalExpansion
from utils.io import write

In [2]:
tensorMapMerged = equistore.io.load('./tensorMapMerged.npz')

In [3]:
frames = ase.io.read("exfoliation_mbd_reference.xyz", ":")

train_energy = np.array([
    frame.info["energy"] for frame in frames
])

train_forces = np.vstack([
    frame.arrays["forces"] for frame in frames
])


import parentParameters as pp
calculator_lr = PyLODESphericalExpansion(pp.hypers_lr)
descriptor_lr = calculator_lr.compute(frames)
inv_lrlr = compute_power_spectrum(descriptor_lr)
inv_lrlr.keys_to_properties(['neighbor_species_1', 'neighbor_species_2'])
inv_lrlr.keys_to_samples('center_species')

In [4]:
print(tensorMapMerged.block(0).has_gradient('positions'))
print('')
print(inv_lrlr.block(0).has_gradient('positions'))

True

True


In [5]:
model_lrlr = LinearModel(normalize=True, regularizer=[1e-4, 5e-4])

model_lrlr.fit(inv_lrlr, train_energy, train_forces)

pred_energy_lrlr, pred_forces_lrlr = model_lrlr.predict(inv_lrlr, with_forces=True)

In [12]:
# for i in np.arange(len(tensorMapMerged.block(0).values)):
#     for j in np.arange(len(tensorMapMerged.block(0).values[i])):
#         if tensorMapMerged.block(0).values[i][j] != inv_lrlr.block(0).values[i][j]:
#             print("Warning")
#         else:
#             continue
            
print(tensorMapMerged.block(0).gradient('positions').data.shape)

print(inv_lrlr.block(0).gradient('positions').data.shape)

# for i in np.arange(tensorMapMerged.block(0).gradient('positions').data.shape[0]):
#     for j in np.arange(tensorMapMerged.block(0).gradient('positions').data.shape[1]):
#         for k in np.arange(tensorMapMerged.block(0).gradient('positions').data.shape[2]):
#             if tensorMapMerged.block(0).gradient('positions').data[i][j][k] != inv_lrlr.block(0).gradient('positions').data[i][j][k]:
#                 print('Warning')
#             else:
#                 continue


structures = np.unique(tensorMapMerged.block(0).samples['structure'])
print(structures)

gradStructures = np.unique(tensorMapMerged.block(0).gradient('positions').samples['structure'])
print(gradStructures)

print(np.all(
    np.unique(tensorMapMerged.block(0).gradient('positions').samples['structure']) == structures
))


(6400, 3, 100)
(6400, 3, 100)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
True
